In [39]:
minu = input("Enter the minuend: ").upper()
subt = input("Enter the subtrahend: ").upper()
diff = input("Enter the difference: ").upper()
print("\n\n%s\n-%s\n------\n %s" % (minu,subt,diff))
print("\n\nIMPLIES THAT...")
print("\n\n %s\n+%s\n------\n%s" % (subt,diff,minu))

DEFAULT_ASGN = 'NA'
FAILURE = True

Enter the minuend:  eight
Enter the subtrahend:  four
Enter the difference:  four




EIGHT
-FOUR
------
 FOUR


IMPLIES THAT...


 FOUR
+FOUR
------
EIGHT


In [94]:
def get_constraints(minu, subt, diff):
    '''Create list of constraints based on user input.
    '''
    minu, subt, diff = map(list, [minu, subt, diff]) # convert strings into lists
    constraints = []
    for i in range(1,len(minu)):
#         constraints.append("C{} in [0,1]".format(i))
        if i > 1:
            constraints.append("C{} + {} + {} == {} + 10 * C{}".format(i-1, diff[-i], subt[-i], minu[-i], i)) 
        else:
            constraints.append("{} + {} == {} + 10 * C{}".format(diff[-i], subt[-i], minu[-i], i))        
    constraints.append("C4 == {}".format(minu[0]))
#     constraints.append("{} > 0".format(minu[0]))
#     constraints.append("{} > 0".format(subt[0]))
#     constraints.append("{} > 0".format(diff[0]))
    return constraints

def get_variables(minu, subt, diff):
    '''Create a dictionary in which the keys are the variables and the values represent the 
    domains for each variable.
    '''
    variables = []
    for ch in minu+subt+diff:
        if ch not in variables:
            variables.append(ch)
    var_doms = dict((el,list(range(10))) for el in variables)
    for i in range(1,len(minu)):
        var_doms["C{}".format(i)] = [0,1]
    return var_doms

def no_leading_zero(minu, subt, diff, var_doms):
    '''Remove the domain value of zero from leading characters.
    '''
    l = set([minu[0], subt[0], diff[0]])
    for el in l:
        var_doms[el].remove(0)
    return var_doms

def final_carry(minu, var_doms):
    '''Create the constraint that the final carry value must equal 1.
    '''
    var_doms[minu[0]] = [1]
    var_doms['C4'] = [1]
    return var_doms

def initialize(minu, subt, diff):
    ''' Create constraints and remove all of the unnecessary domain values from the 
    each variable's list of possible domain values.
    '''
    constraints = get_constraints(minu, subt, diff)
    var_doms = get_variables(minu, subt, diff)
    var_doms = no_leading_zero(minu, subt, diff, var_doms)
    var_doms = final_carry(minu, var_doms)
    return var_doms, constraints

def backtrack(asgns, var_doms, ctr, constraints):
    '''The actual backtrack search algorithm.
    '''
    print("Iteration #"+str(ctr))
    if DEFAULT_ASGN not in list(asgns.values()):
        print(ctr, asgns)
        return asgns
    # find first variable in asgns that hasn't yet been assigned a value
    cur_var = [k for k, v in asgns.items() if v == DEFAULT_ASGN][0]
    for val in var_doms[cur_var]:
        if unused(val, asgns) and is_consistent(cur_var, val, asgns, constraints):
            asgns[cur_var] = val
            result = backtrack(asgns, var_doms, ctr+1, constraints)
            if result != False:
                return result
        asgns[cur_var] = DEFAULT_ASGN
    return False

def is_consistent(test_var, test_val, asgns, constraints):
    ''' For each of the variable names, replace every occurence of the variable in the list of 
    constraints with the value that that variable was assigned to in the asgns dictionary.
    '''
    for var in list(asgns.keys()):
        if asgns[var] != 'NA':
            for constraint in constraints:
                if var != test_var:
                    # replace variable with value
                    constraint.replace(var, str(asgns[var]))
                else:
                    constraint.replace(var, str(test_val))
    # check to see if constraints fail to evaluate to True
    for constraint in constraints:
        try:
            # test each constraint
            exec('satisfied = {}'.format(constraint))
        except NameError:
            satisfied = True
        # if any constraint fails, ie evaluates to False, return False
        if not satisfied:
            return False
    return True

def unused(test_val, asgns):
    '''Returns True if all of the variable assignments are unique.
    '''
#     values = list(filter((DEFAULT_ASGN).__ne__, list(asgns.values())))
#     return len(values) == len(set(values))
    return test_val not in asgns.values()

In [99]:
var_doms, constraints = initialize(minu, subt, diff)

asgns = dict((el, 'NA') for el in list(var_doms.keys()))

solution = backtrack(asgns, var_doms, 0, constraints)

print(constraints)

Iteration #0
Iteration #1
Iteration #1
Iteration #2
Iteration #3
Iteration #4
Iteration #5
Iteration #5
Iteration #5
Iteration #5
Iteration #5
Iteration #5
Iteration #4
Iteration #5
Iteration #5
Iteration #5
Iteration #5
Iteration #5
Iteration #5
Iteration #4
Iteration #5
Iteration #5
Iteration #5
Iteration #5
Iteration #5
Iteration #5
Iteration #4
Iteration #5
Iteration #5
Iteration #5
Iteration #5
Iteration #5
Iteration #5
Iteration #4
Iteration #5
Iteration #5
Iteration #5
Iteration #5
Iteration #5
Iteration #5
Iteration #4
Iteration #5
Iteration #5
Iteration #5
Iteration #5
Iteration #5
Iteration #5
Iteration #4
Iteration #5
Iteration #5
Iteration #5
Iteration #5
Iteration #5
Iteration #5
Iteration #3
Iteration #4
Iteration #5
Iteration #5
Iteration #5
Iteration #5
Iteration #5
Iteration #5
Iteration #4
Iteration #5
Iteration #5
Iteration #6
Iteration #5
Iteration #6
Iteration #5
Iteration #6
Iteration #5
Iteration #6
Iteration #5
Iteration #6
Iteration #4
Iteration #5
Iteration #5

In [100]:
var_doms

{'C1': [0, 1],
 'C2': [0, 1],
 'C3': [0, 1],
 'C4': [1],
 'E': [1],
 'F': [1, 2, 3, 4, 5, 6, 7, 8, 9],
 'G': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'H': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'I': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'O': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'R': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'T': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'U': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}

In [103]:
asgns['C4'] = 1
asgns['E'] = 1
asgns

{'C1': 'NA',
 'C2': 'NA',
 'C3': 'NA',
 'C4': 1,
 'E': 1,
 'F': 'NA',
 'G': 'NA',
 'H': 'NA',
 'I': 'NA',
 'O': 'NA',
 'R': 'NA',
 'T': 'NA',
 'U': 'NA'}

In [102]:
constraints

['R + R == T + 10 * C1',
 'C1 + U + U == H + 10 * C2',
 'C2 + O + O == G + 10 * C3',
 'C3 + F + F == I + 10 * C4',
 'C4 == E']

In [105]:
[[k,v] for k, v in asgns.items() if 'C' not in k]

[['F', 'NA'],
 ['E', 1],
 ['T', 'NA'],
 ['H', 'NA'],
 ['O', 'NA'],
 ['I', 'NA'],
 ['R', 'NA'],
 ['G', 'NA'],
 ['U', 'NA']]